## Introduction

This project utilizes Google Cloud Functions and Cloud Scheduler to automate the daily tracking of YouTube and Spotify metrics for an artist. By streamlining data collection, processing, and storage, it eliminates the need for manual intervention, making it easier for music industry professionals to monitor video performance and audience engagement. The processed metrics are stored in a Google Sheet, providing a centralized platform for evaluating an artist's reach and popularity.

## Objective

The primary goal of this project is to build an automated pipeline that leverages Google Cloud Scheduler for daily, real-time updates. The function collects critical metrics, including subscriber count, views, likes, and comments for the top 10 YouTube videos (excluding Shorts), along with Spotify metrics like follower count and popularity. This data is processed into a unified format and replaces previous records in a Google Sheet to maintain accuracy and eliminate redundancy. This automated workflow delivers actionable insights, empowering users to track performance and optimize engagement strategies efficiently.

### 1. Deochii Daily Daily Metrics

#### Testing the cloud function

In [1]:
import requests

# Cloud Function URL
cloud_function_url = "https://us-central1-linear-bounty-441500-b2.cloudfunctions.net/deochii_daily_metrics"

def test_cloud_function():
    # Send an HTTP POST request to trigger the function
    response = requests.post(cloud_function_url)
    
    # Check the response status code
    if response.status_code == 200:
        print("\033[1;34mCloud function executed successfully!\033[0m")
        print("Response:", response.text)
    else:
        print("Error:", response.status_code)
        print("Response:", response.text)

# Call the function to test
test_cloud_function()

Cloud function executed successfully!
Response: Data stored successfully in Google Sheets!


## Deconstructing the functions 

These collection of functions automate the collection and storage of YouTube and Spotify metrics for an artist:

1. **Retrieve Channel Info**: Fetches the YouTube channel ID and subscriber count.
2. **Fetch Top Videos**: Retrieves the top 10 YouTube videos (excluding Shorts) and metrics like views, likes, and comments.
3. **Spotify Data**: Gets Spotify popularity and follower count for the artist.
4. **Update Google Sheets**: Replaces old data in a Google Sheet with the latest metrics.

### *Step 1* : Extracting channel ID and subscriber count from YouTube username

In [2]:
def get_channel_info(api_key, username):
    
    url = "https://www.googleapis.com/youtube/v3/search"
    params = {
        "part": "snippet",
        "q": username,
        "type": "channel",
        "key": api_key
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()
    logging.info("YouTube API request successful for username: %s", username)
    
    items = response.json().get("items", [])
    
    channel_id = items[0]["snippet"]["channelId"]
    subscriber_count = fetch_youtube_subscribers(api_key, channel_id)
    logging.info("Subscriber count retrieved for channel ID %s: %d", channel_id, subscriber_count)
    
    return channel_id, subscriber_count

### *Step 2* : To fetch top 10 Youtube videos by view count and metrics

In [3]:
def fetch_top_10_youtube_videos(api_key, channel_id):
    try:
        logging.info("Fetching top 10 videos from YouTube channel")
        url = "https://www.googleapis.com/youtube/v3/search"
        params = {
            "part": "snippet",
            "channelId": channel_id,
            "order": "viewCount",
            "type": "video",
            "maxResults": 50,
            "key": api_key
        }
        
        videos, count, next_page_token = [], 1, None

        while count <= 10:
            if next_page_token:
                params["pageToken"] = next_page_token

            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()

            for item in data.get("items", []):
                video_id = item["id"].get("videoId")
                if video_id and "shorts" not in item["snippet"]["title"].lower():
                    video_stats = fetch_youtube_video_stats(api_key, video_id)
                    videos.append({
                        "Rank": count,
                        "Title": item["snippet"]["title"],
                        "Artist Name": item["snippet"]["channelTitle"],
                        "Views": video_stats["viewCount"],
                        "Likes": video_stats["likeCount"],
                        "Comments": video_stats["commentCount"],
                        "Video ID": video_id
                    })
                    count += 1
                    if count > 10:
                        break

            next_page_token = data.get("nextPageToken")
            if not next_page_token:
                break

        return videos
    except Exception as e:
        logging.error("Error fetching top YouTube videos: %s", e)
        return []

def fetch_youtube_video_stats(api_key, video_id):
    try:
        url = "https://www.googleapis.com/youtube/v3/videos"
        params = {"part": "statistics", "id": video_id, "key": api_key}
        response = requests.get(url, params=params)
        response.raise_for_status()
        stats = response.json()["items"][0]["statistics"]
        return {
            "viewCount": int(stats.get("viewCount", 0)),
            "likeCount": int(stats.get("likeCount", 0)),
            "commentCount": int(stats.get("commentCount", 0))
        }
    except Exception:
        logging.error("Error fetching video statistics for video ID: %s", video_id)
        return {"viewCount": 0, "likeCount": 0, "commentCount": 0}


### *Step 3* : Accessing Spotify metrics

In [5]:
def get_spotify_token():
    try:
        client_id, client_secret = os.getenv("CLIENT_ID"), os.getenv("CLIENT_SECRET")
        if not client_id or not client_secret:
            raise ValueError("Missing Spotify CLIENT_ID or CLIENT_SECRET")

        url = "https://accounts.spotify.com/api/token"
        headers = {"Authorization": "Basic " + base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()}
        response = requests.post(url, headers=headers, data={"grant_type": "client_credentials"})
        response.raise_for_status()
        return response.json().get("access_token")
    except Exception as e:
        logging.error("Error getting Spotify token: %s", e)
        return None

def fetch_spotify_artist_data(token, artist_name):
    try:
        url = "https://api.spotify.com/v1/search"
        headers = {"Authorization": f"Bearer {token}"}
        params = {"q": artist_name, "type": "artist", "limit": 1}
        
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        
        artist = response.json()["artists"]["items"][0]
        return {"Spotify Popularity": artist["popularity"], "Followers": artist["followers"]["total"]}
    except Exception as e:
        logging.error("Error fetching Spotify artist data for %s: %s", artist_name, e)
        return {"Spotify Popularity": 0, "Followers": 0}

### *Step 4* : Storing data in a Google Sheet

In [ ]:
def store_to_google_sheets(data, subscribers, sheet_name="Doechii Daily Metrics"):
    try:
        logging.info("Authenticating with Google Sheets API")
        scope = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
        creds = service_account.Credentials.from_service_account_file("service_account_key.json", scopes=scope)
        client = gspread.authorize(creds)
        
        logging.info(f"Opening Google Sheet: {sheet_name}")
        sheet = client.open(sheet_name).sheet1

        # Insert each artist's data at the top (after headers)
        for artist_data in reversed(data):  # Reverse to keep rank order
            new_row = [
                artist_data["Rank"],
                artist_data["Title"],
                artist_data["Artist Name"],
                artist_data["Views"],
                artist_data["Likes"],
                artist_data["Comments"],
                subscribers,  # Add YouTube subscribers
                artist_data.get("Spotify Popularity", "N/A"),
                artist_data.get("Followers", "N/A"),
                datetime.now().strftime("%Y-%m-%d"),
                datetime.now().strftime("%H:%M:%S")
            ]
            
            logging.info("Inserting data into Google Sheets at the top: %s", new_row)
            sheet.insert_row(new_row, index=2)
        
        logging.info("Data successfully inserted into Google Sheets at the top")


### *Step 5* : Final Function 

In [6]:
def store_to_google_sheets(data, subscribers, sheet_name="Doechii Daily Metrics"):
    try:
        creds = service_account.Credentials.from_service_account_file(
            "service_account_key.json",
            scopes=["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
        )
        sheet = gspread.authorize(creds).open(sheet_name).sheet1

        for artist_data in reversed(data):  # Insert in reverse to preserve rank order
            sheet.insert_row([
                artist_data["Rank"],
                artist_data["Title"],
                artist_data["Artist Name"],
                artist_data["Views"],
                artist_data["Likes"],
                artist_data["Comments"],
                subscribers,
                artist_data.get("Spotify Popularity", "N/A"),
                artist_data.get("Followers", "N/A"),
                datetime.now().strftime("%Y-%m-%d"),
                datetime.now().strftime("%H:%M:%S")
            ], index=2)
        logging.info("Data successfully inserted into Google Sheets")
    except Exception as e:
        logging.error("Error storing data to Google Sheets: %s", e)